# 声紋認証用

In [47]:
import urllib.request
import base64
import urllib
import xml.etree.ElementTree as ET

# Basic認証の情報
user = "<user_name>"
password = "<user_pass>"
configName = "<config_Name>"
serverIp = "<server_IP>"

def nuance_recog(target_user, voiceprintTag, audio_path):    
    session_id = ""
    audio_segment_id = ""
    audioFile = ""
    requestId = ""
    # BiometricRiskLevel
    bio_risk_level = 0

    # Basic認証用の文字列を作成.
    basic_user_and_pasword = base64.b64encode('{}:{}'.format(user, password).encode('utf-8'))
    
    #
    # StartSession
    #

    url = 'http://' + serverIp + '/SecuritySuite/FreeSpeechServer.asmx/StartSession'

    # Basic認証付きの、GETリクエストを作成する.
    text = {"configSetName": configName,
           "externalSessionId": ""}
    data = urllib.parse.urlencode(text).encode("utf-8")
    # print(data)
    request = urllib.request.Request(url, 
        headers={"Authorization": "Basic " + basic_user_and_pasword.decode('utf-8'),
                "content-type": "application/x-www-form-urlencoded"},
        data = data)
    # 送信して、レスポンスを受け取る.
    with urllib.request.urlopen(request) as res:
        response = res.read()
    # print(response)
    root = ET.fromstring(response)

    session_id = root[0].text

    print("session_id: {}".format(session_id))
    
    
    #
    # StartAudio
    #
    
    url = 'http://' + serverIp + '/SecuritySuite/FreeSpeechServer.asmx/StartAudio'

    # Basic認証付きの、GETリクエストを作成する.
    text = {"sessionId": session_id,
           "configSetName": configName}
    data = urllib.parse.urlencode(text).encode("utf-8")

    request = urllib.request.Request(url, 
        headers={"Authorization": "Basic " + basic_user_and_pasword.decode('utf-8'),
                "content-type": "application/x-www-form-urlencoded"},
        data = data)
    # 送信して、レスポンスを受け取る.
    with urllib.request.urlopen(request) as res:
        response = res.read()
    #     print(response)

    root = ET.fromstring(response)
    audio_segment_id = root.text

    print("audio_segment_id: {}".format(audio_segment_id))
    
    
    #
    # AddAudio
    #

    url = 'http://' + serverIp + '/SecuritySuite/FreeSpeechServer.asmx/AddAudio'

    with open(audio_path, 'rb') as f:
        audioFile = base64.b64encode(f.read())

    # Basic認証付きの、GETリクエストを作成する.
    text = {"sessionId": session_id,
            "audioSegmentId": audio_segment_id,
            "audio":audioFile,
           "configSetName": configName}
    data = urllib.parse.urlencode(text).encode("utf-8")

    request = urllib.request.Request(url, 
        headers={"Authorization": "Basic " + basic_user_and_pasword.decode('utf-8'),
                "content-type": "application/x-www-form-urlencoded"},
        data = data)
    # 送信して、レスポンスを受け取る.
    with urllib.request.urlopen(request) as res:
        # 特に取得したい情報はないため、ステータスコードを取得
        response = res.getcode()
        print("add_audio.status: {}".format(response))

        
    #
    # Verify
    #
    
    url = 'http://' + serverIp + '/SecuritySuite/FreeSpeechServer.asmx/Verify'

    # Basic認証付きの、GETリクエストを作成する.
    text = {"sessionId": session_id,
            "speakerId": target_user,
            "voiceprintTag": voiceprintTag,
            "audio": audio_segment_id,
           "configSetName": configName}
    data = urllib.parse.urlencode(text).encode("utf-8")
    print(data)
    request = urllib.request.Request(url, 
        headers={"Authorization": "Basic " + basic_user_and_pasword.decode('utf-8'),
                "content-type": "application/x-www-form-urlencoded"},
        data = data)
    # 送信して、レスポンスを受け取る.
    with urllib.request.urlopen(request) as res:
        response = res.read()
        print(response)
        root = ET.fromstring(response)
        requestId = root[0].text

    print("requestId: {}".format(requestId))
    

    #
    # GetDetailedResult
    #
    
    url = 'http://' + serverIp + '/SecuritySuite/FreeSpeechServer.asmx/GetDetailedResult'

    # Basic認証付きの、GETリクエストを作成する.
    text = {"sessionId": session_id,
            "requestId": requestId,
           "configSetName": configName}
    data = urllib.parse.urlencode(text).encode("utf-8")

    request = urllib.request.Request(url, 
        headers={"Authorization": "Basic " + basic_user_and_pasword.decode('utf-8'),
                "content-type": "application/x-www-form-urlencoded"},
        data = data)
    # 送信して、レスポンスを受け取る.
    with urllib.request.urlopen(request) as res:
        response = res.read()
        print(response)
    root = ET.fromstring(response)

    target_user = root[7][0][0].text
    decision = root[0][2].text
    bio_risk_level = float(root[7][0][10].text)
    confidence = 100 - bio_risk_level
    
    print("----------")
    print("対象データ: {}".format(audio_path))
    print("ユーザID: {}".format(target_user))
    print("判定: {}".format(decision))
    print("本人である確度: {}%".format(confidence))
#     print("BiometricRiskLevel: {}%".format(bio_risk_level))
    
    
    return confidence
